In [1]:
! pip install -U spacy -q 

     |████████████████████████████████| 6.0 MB 5.5 MB/s 
     |████████████████████████████████| 451 kB 70.7 MB/s 
     |████████████████████████████████| 10.1 MB 49.4 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 181 kB 57.0 MB/s 
     |████████████████████████████████| 628 kB 60.3 MB/s 


In [2]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.2.1                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.12                        
Pipelines                                      



In [5]:
# !wget https://raw.githubusercontent.com/amrrs/custom-ner-with-spacy/main/pvr_training_data.json

--2021-12-30 17:13:03--  https://raw.githubusercontent.com/amrrs/custom-ner-with-spacy/main/pvr_training_data.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2829 (2.8K) [text/plain]
Saving to: ‘pvr_training_data.json’

pvr_training_data.j 100%[===================>]   2.76K  --.-KB/s    in 0s      

2021-12-30 17:13:03 (58.1 MB/s) - ‘pvr_training_data.json’ saved [2829/2829]



In [2]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("pt") # load a new spacy model
db = DocBin() # create a DocBin object

In [3]:
import json
with open('training_data.json', encoding='utf-8') as f:
    TRAIN_DATA = json.load(f)

In [4]:
for text, annot in tqdm(TRAIN_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

100%|██████████| 497/497 [00:00<00:00, 1425.77it/s]

Skipping entity


In [5]:
! python -m spacy init config config.cfg --lang pt --pipeline ner --optimize efficiency

ℹ Generated config template specific for your use case
- Language: pt
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [6]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     43.45    0.00    0.00    0.00    0.00
  0     200       1281.09   3453.44   47.35   64.94   37.26    0.47
  0     400        133.29   1311.26   52.82   61.04   46.55    0.53
  1     600         89.55   1165.79   61.36   62.67   60.10    0.61
  1     800         94.26   1222.41   62.27   64.11   60.54    0.62
  2    1000        228.12   1274.77   68.10   68.59   67.62    0.68
  3    1200        397.68   1381.33   67.58   67.03   68.13    0.68
  3    1400       1969.06   1662.59   72.45   85.31   62.95    0.72
  4    1600        269.68   1560.74 

In [3]:
nlp_ner = spacy.load("model-best") 

In [8]:
doc = nlp_ner('''Comprei o notebook em junho de 2024, veio lacrado, tudo direitinho. Já tive outro notebook da positivo, que me serviu muito bem por mais de 10 anos, por isso preferi comprar da mesma marca. Comprei pois estava precisando p estudar, com urgência! Já não fiquei muito satisfeita pq achei muito lento, mas tudo bem. Com uns 3 meses deu defeito, e simplesmente não ligou mais, conectava no carregador, mas não carregava. Levei p autorizada, já tem 1 mês, e nada! A gente faz uma dívida, sem poder, por necessidade, ainda estou pagando, além de já não ter ficado tão satisfeita c o produto, ainda tem, ainda tem esse tipo de problema''') # input sample text

In [9]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter